<a href="https://colab.research.google.com/github/alejandromanas/GoogleColab_public/blob/master/Bookings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Bookings


Libraries

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn import metrics

### Loading Files from Drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
X = pd.read_csv("/content/drive/My Drive/colab files/bookings/X_xg_model.csv", delimiter = ',')
Y = pd.read_csv("/content/drive/My Drive/colab files/bookings//Y_xg_model.csv", delimiter = ',')

### Preprocessing

In [0]:
df_res=raw_data_res.copy()
df_urb=raw_data_urb.copy() 
        
        #Concateno valores de los dos hoteles en un unico dataframe
        #creo nueva columna Hotel
        
df_res['Hotel']='Resort'
df_urb['Hotel']='Urban'
        
df_all=pd.concat([df_res,df_urb], axis=0)
df_all = df_all.reset_index()
del df_all['index']
        
   
    
    ##########################
    # IMPORT DATA
    ##########################
    
   # list(df_all.columns)
    
    ##########################
    # NAN
    ##########################
    
    #Buscamos nan
    # df_all.isnull().values.any() #Hay algun nan
    # df_all.isnull() #matriz logica para bsucarlos
    # df_all.isnull().any() #en que columna esta el nan
    
    #Hotel urbano existen NA en columna children
    #SN cuatro reservas canceladas con campo Children NA. 
    
    
    
    #CHILDREN
df_all['Children'] = df_all.Children.replace(np.nan, 0)
df_all['Children'] = df_all.Children.astype('int')
    
    # vtreat

X_vtreat=pd.DataFrame()
Y_vtreat=pd.DataFrame()
    # X_test=df_model.loc[:, df_model.columns != 'IsCanceled'].copy()
X_vtreat=df_all[['Company','Agent']].copy()
Y_vtreat=df_all['IsCanceled'].copy()
transform = vtreat.BinomialOutcomeTreatment(
  outcome_name='IsCanceled',    # outcome variable
  outcome_target=True,  # outcome of interest
  cols_to_copy=['X_vtreat'],   # columns to "carry along" but not treat as input variables
      )

    # plan = vtreat.BinomialOutcomeTreatment(outcome_target=True)
d_prepared = transform.fit_transform(X_vtreat,df_all['IsCancelled'])


KeyError: ignored

In [0]:
l_columns = list(df.columns) # show column names as list
for i in l_columns:
  print(' column :',i,'type :',df[i].dtype)
  print(df[i].value_counts(dropna=False))
  count.df[i].value_counts(dropna=False)

 column : index type : int64
2047     2
19603    2
21206    2
19157    2
17108    2
        ..
64069    1
62020    1
51779    1
49730    1
59944    1
Name: index, Length: 79330, dtype: int64
 column : IsCanceled type : int64
0    75166
1    44224
Name: IsCanceled, dtype: int64
 column : LeadTime type : int64
0      6345
1      3460
2      2069
3      1816
4      1715
       ... 
458       1
371       1
737       1
435       1
387       1
Name: LeadTime, Length: 479, dtype: int64
 column : ArrivalDateYear type : int64
2016    56707
2017    40687
2015    21996
Name: ArrivalDateYear, dtype: int64
 column : ArrivalDateMonth type : object
August       13877
July         12661
May          11791
October      11160
April        11089
June         10939
September    10508
March         9794
February      8068
November      6794
December      6780
January       5929
Name: ArrivalDateMonth, dtype: int64
 column : ArrivalDateWeekNumber type : int64
33    3580
30    3087
32    3045
34    3040
18  

In [0]:
Y_vtreat.shape

(119390,)

In [0]:
df.columns

Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'ReservedRoomType', 'AssignedRoomType',
       'BookingChanges', 'DepositType', 'Agent', 'Company',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'ReservationStatus', 'ReservationStatusDate',
       'hotel_type'],
      dtype='object')

In [0]:
df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            79320, 79321, 79322, 79323, 79324, 79325, 79326, 79327, 79328,
            79329],
           dtype='int64', length=119390)

In [0]:
df.index


RangeIndex(start=0, stop=119390, step=1)

### XGBOOST

In [0]:
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    time_elapsed=print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

def XGBC_Boss(X,Y):
    
    # time 
    _start_time = time.time()
    
    tic()
    
    # Model
    
    xgbmodel=XGBClassifier()
    
    xgbmodel.get_params()
    
    params={'base_score': [0.8], # prediccion inicial
         'booster': ['gbtree'], # (gbtree, gblinear, dart)
         'colsample_bylevel': [1], # ratio de columnas en cada nivel
         'colsample_bytree': [0.8], # ratio de columnas por tree
         'gamma': [0.01],    # minimo "loss" reduccion para crear un nuevo split. Larger-> conservative
         'learning_rate': [0.01], # (eta) aportacion de cada arbol al modelo
         'max_depth': [6], # maxima profundidad en cada arbol
         'min_child_weight': [1], # minimo numero samples por hoja
         'missing': [None], # si queremos reemplazar los missings por un numero
         'n_estimators': [400], # numero de arboles
         'n_jobs': [1], # trabajos en paralelo
         'objective': ['binary:logistic'],#  Output. Tipo de función que estamos estimando
         'random_state': [15], # seed para generar los folds
         'reg_alpha': [0.01], # L1 regularitacion
         'reg_lambda': [0.01], # L2 regularitacion
         'scale_pos_weight': [1],
         'subsample': [0.9]} # ratio de muestras por cada arbol 
    
    
    # kfold = model_selection.StratifiedKFold(n_splits=3, random_state=15)
    kfold = model_selection.StratifiedShuffleSplit(n_splits=3, random_state=15)
    grid_solver = GridSearchCV(estimator = xgbmodel, # model to train
                       param_grid = params, # param_grid
                       scoring = ['accuracy'],
                       cv = kfold,
                        n_jobs=3, 
                       # n_jobs=-1,
                       refit = 'accuracy',
                       verbose = 2)
    
    model_result_xgboost = grid_solver.fit(X,Y)
    # model_result_xgboost.best_estimator_
    # X.shape
    
    # return predictions
    Y_pred = pd.Series(model_result_xgboost.predict(X))
    Y_prob= model_result_xgboost.predict_proba(X)
    # Y_prob.shape
    # Y.shape
    
    # Metrics and parameters
    
    print('Best parameters',str(model_result_xgboost.best_params_))
    results=pd.DataFrame(model_result_xgboost.cv_results_)
     
    print('Score Acc',str(model_result_xgboost.score(X,Y)))
    
    fpr, tpr, threshold = metrics.roc_curve(Y, Y_prob[:,1])
    roc_auc = metrics.auc(fpr, tpr)
    print('AUC = %0.2f' % roc_auc)
    
    Logloss = metrics.log_loss(Y, Y_prob[:,1])
    print('LogLoss = %0.2f' % Logloss)
    
    # model_result_xgboost.predict(X)
    xgbm=model_result_xgboost.best_estimator_
    
    # Plot importance PENDING
    fig, ax = plt.subplots(figsize=(20, 15))
    xgb.plot_importance(xgbm,ax=ax)
    # fig = plot_imp.figure
    # fig.set_size_inches(5, 10)
    plt.savefig('XG_Boost.png')

    #pickle.dump(model_result_xgboost, open("model_result_xgboost.pickle.dat", "wb"))
    
    tac() # time elapsed
    
  
    return Y_pred,Y_prob,model_result_xgboost 

XGBC_Boss(X,Y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  3.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### lightGBM

In [0]:
def light_LGBM(X,y,param_grid):
    
  _start_time = time.time()
  tic()
     
  indexes_of_categories=[]
    
  lbl = preprocessing.LabelEncoder()
    
  for i in range(X.shape[1]):
      if X.dtypes[i]=='object':
          indexes_of_categories.append(i)
          lbl.fit(X.iloc[:,i])
          X.iloc[:,i]=lbl.transform(X.iloc[:,i])
        
  # Splitting
           
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)            
          
  # gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=train, y=y)
            
  gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X_train, y=y_train)
    
   
  # lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=indexes_of_categories)
    
   
  lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary',num_boost_round=2000)
    
  print('Fitting...')
    
  scoring = ['roc_auc']
  refit='roc_auc'
  # scoring = ['accuracy']
  # refit='accuracy'
   
  gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf,scoring=scoring,refit=refit)
    
  lgb_model = gsearch.fit(X=X_train, y=y_train)
    


  print('Results ********** ')
    
  print('Best score auc: '+ str(lgb_model.best_score_))
    
  print('Best parameters:',str(lgb_model.best_params_))
  print('Results :',str(lgb_model.cv_results_))
  # results=pd.DataFrame(lgb_model.cv_results_)
    
  print('********** TRAIN ********** ')
    
  predict_metrics(X_train,y_train,lgb_model)
        
  # # return predictions TRAIN
  # y_train_pred=pd.DataFrame()
  # y_train_pred = pd.Series(lgb_model.predict(X_train))
  # y_train_prob= lgb_model.predict_proba(X_train) 
    
  # print('Score Acc',str(metrics.accuracy_score(y_train,y_train_pred)))
    
  # fpr, tpr, threshold = metrics.roc_curve(y_train, y_train_prob[:,1])
  # roc_auc = metrics.auc(fpr, tpr)
  # print('AUC = %0.2f' % roc_auc)
    
  # Logloss = metrics.log_loss(y_train, y_train_prob[:,1])
  # print('LogLoss = %0.2f' % Logloss)
    
  print('********** TEST ********** ')
  predict_metrics(X_test,y_test,lgb_model)
  # # return predictions TEST
  # y_test_pred=pd.DataFrame()
  # y_test_pred = pd.Series(lgb_model.predict(X_test))
  # y_test_prob= lgb_model.predict_proba(X_test) 
    
  # print('Score Acc',str(metrics.accuracy_score(y_test,y_test_pred)))
    
  # fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_prob[:,1])
  # roc_auc = metrics.auc(fpr, tpr)
  # print('AUC = %0.2f' % roc_auc)
    
  # Logloss = metrics.log_loss(y_test, y_test_prob[:,1])
  # print('LogLoss = %0.2f' % Logloss)
    
  print('********** ALL DATA ********** ')
  y_pred,y_prob=predict_metrics(X,y,lgb_model)
  # return predictions TEST
  # y_test_pred=pd.DataFrame()
  # y_pred = pd.Series(lgb_model.predict(X))
  # y_prob= lgb_model.predict_proba(X) 
    
  # print('Score Acc',str(metrics.accuracy_score(y,y_pred)))
    
  # fpr, tpr, threshold = metrics.roc_curve(y, y_prob[:,1])
  # roc_auc = metrics.auc(fpr, tpr)
  # print('AUC = %0.2f' % roc_auc)
    
  # Logloss = metrics.log_loss(y, y_prob[:,1])
  # print('LogLoss = %0.2f' % Logloss)
   
  # Plot importance PENDING
  # fig, ax = plt.subplots(figsize=(20, 15))
    
  # lgb.plot_importance(lgb_estimator,ax=ax)
  # lgb.plot_importance(gsearch.Booster,ax=ax)
  # lgb.plot_importance(lgb_model.Booster_Booster,ax=ax)
  # lgb.plot_importance(lgb_model,ax=ax)
    
  # plot_tree(lgb_model._Booster)
    
    
  # lgb_model.plot_importance()
  # # fig = plot_imp.figure
  # # fig.set_size_inches(5, 10)
  # plt.savefig('lightLGBM.png')
        
   
  # time
  tac()
    
  return y_pred,y_prob,

param_grid = {
'num_leaves': [31],
  # 'num_boost_round': [2000],
  'learning_rate':[0.1],
  # 'reg_alpha': [0.1],
  'min_data_in_leaf': [30],
  'lambda_l1': [0],
  'lambda_l2': [0],
  'min_data_in_leaf':[100],
  'max_depth': [3],
  # 'num_boost_round': [2000]
  # n_estimators=100,
  # subsample=1.0, 
  }    

y_pred,y_prob = light_LGBM(X,y,param_grid)